In [1]:
'''
交叉验证 做训练集
'''
import pandas as pd
import csv
# read CSV file directly from a URL and save the results
# NOT: First extract zip file
#data = pd.read_csv('NISM.csv',   quoting=csv.QUOTE_NONE, encoding='utf-8')
data = pd.read_csv('ourdataset_test_small4.csv',   quoting=csv.QUOTE_NONE, encoding='utf-8')

#families whose count is greater than 30 
#class_group = df.groupby("class")

print(data.groupby('label').size())

# extact features and labels
# X feature set, y labels
X = data.loc[:,data.columns[:-1]] # -1 label kolonunu refer ediyor
y = data.label
X = X.values

print(X.shape)
# print X
# print '----------------------------------------------'

# Generally, this is one of the most commonly used preprocessing step.特征值预处理
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn import preprocessing
#数据归一化 和 标准化
# normalize the data attributes  实验发现效果不理想 因此不做归一化
# normalized_X = preprocessing.normalize(X)
# standardize the data attributes 数据标准化 效果拔群
standardized_X = preprocessing.scale(X)
X = standardized_X

#下面的树算法(Tree algorithms)计算特征的信息量：
clf = ExtraTreesClassifier()
clf = clf.fit(X, y)
#print(clf.feature_importances_)  

model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)
X=X_new
print(X.shape)
# print X

from sklearn import linear_model, ensemble, svm, tree, neural_network, metrics, naive_bayes
from sklearn import cross_validation

#训练
def train_flow_classification(X, y, clf_class, shuffle=True, n_folds=10, **kwargs):
#     stratified_k_fold = cross_validation.StratifiedKFold(y, n_folds=n_folds, shuffle=shuffle)
#     y_pred = y.copy()
#     print stratified_k_fold
    clf = clf_class(**kwargs)
    clf.fit(X,y)
#     for ii, jj in stratified_k_fold:
#         X_train, X_test = X[ii], X[jj]
#         y_train = y[ii]
#         #这里的 clf应该放在 for循环的外面，不然 分类器每次for循环都重新训练了
# #         clf = clf_class(**kwargs)
#         clf.fit(X_train,y_train)
    return clf

#预测验证
def pred_flow_classification(X, y, clf, shuffle=True, n_folds=10, **kwargs):
    stratified_k_fold = cross_validation.StratifiedKFold(y, n_folds=n_folds, shuffle=shuffle)
    y_pred = y.copy()
    clf_train = clf
    print stratified_k_fold
#     print y_pred
    for ii, jj in stratified_k_fold:
        X_train, X_test = X[ii], X[jj]
        y_train = y[ii]
        y_pred[jj] = clf_train.predict(X_test)
#         print 'y_pred    ',y_pred[jj]
    return y_pred

# cross validation of the ML algorithms
def stratified_cv(X, y, clf_class, shuffle=True, n_folds=10, **kwargs):
    stratified_k_fold = cross_validation.StratifiedKFold(y, n_folds=n_folds, shuffle=shuffle)
    y_pred = y.copy()
    print stratified_k_fold
    #建议 clf = clf_class(**kwargs) 放在 for循环外面
#     print y_pred
    # ii+jj 的长度是样本总数，而 ii 表示用于训练的样本的序号集合，jj表示用于测试/预测的样本的序号集合
    for ii, jj in stratified_k_fold:
        X_train, X_test = X[ii], X[jj]
        y_train = y[ii]
        #这里的 clf应该放在 for循环的里面，每次for循环分类器都用于重新训练，因为用到了交叉验证，它划分了10份样本，每次拿其中一份做预测样本
        #其余九份做训练，以此来对预测的那一份样本做预测和比较，经过10次for循环所有的10份样本都会被用来交叉验证，最终预测的样本总数等于原样本
        #这样的好处是没有拿所有样本做训练集，而是每次只取其中九份做训练，留一份做预测，一是防止过拟合二是增加实验可信度
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[jj] = clf.predict(X_test)
#         print 'y_pred    ',y_pred[jj]
    return y_pred

# print accuracies of the ML algo 其他机器学习算法的 分类准确率分数 比较，分类准确率分数是指所有分类正确的百分比
# print('Random Forest Classifier:       {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(X, y, ensemble.RandomForestClassifier))))
# print('Support vector machine(SVM):   {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(X, y, svm.SVC))))
# print('Logistic Regression:            {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(X, y, linear_model.LogisticRegression))))
# print('Native Network:            {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(X, y, neural_network.MLPClassifier, activation ='relu',
#                                                                                         hidden_layer_sizes =(30,30,30,30,30,30,30,30,30)))))
#所有样本做训练集
# clf_train = train_flow_classification(X, y, neural_network.MLPClassifier, activation ='relu',hidden_layer_sizes =(30,30,30,30,30,30,30,30,30))
#预测得到的样本 y_pred
# y_pred = pred_flow_classification(X, y,clf_train)
#分类准确率分数
#交叉验证方式 做训练集 预测得到的样本y_pred
y_pred = stratified_cv(X, y, neural_network.MLPClassifier, activation ='relu',hidden_layer_sizes =(30,30,30,30,30,30,30,30,30))
print('Native Network:            {:.2f}'.format(metrics.accuracy_score(y, y_pred)))

#混淆(xiao)矩阵
print '混淆矩阵为 = \n',metrics.confusion_matrix(y, y_pred, labels = ["mail","skype","ssh","teamviewer","telegram"])
print 'finish'

label
mail          10780
skype          9835
ssh           11165
teamviewer    11760
telegram       9765
dtype: int64
(53305L, 22L)


E:\ProgramData\Anaconda2\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


(53305L, 9L)
sklearn.cross_validation.StratifiedKFold(labels=[u'mail' u'mail' u'mail' ..., u'telegram' u'telegram' u'telegram'], n_folds=10, shuffle=True, random_state=None)


E:\ProgramData\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Native Network:            1.00
混淆矩阵为 = 
[[10759    21     0     0     0]
 [   34  9801     0     0     0]
 [    0     0 11165     0     0]
 [    0     0     0 11760     0]
 [    0     0     0     0  9765]]
finish
